In [2]:
!apt install tesseract-ocr
!pip install pytesseract


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (5,200 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123069 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-eng_1%3

In [3]:
pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import cv2
import pytesseract
from PIL import Image

# Load the image
image_path = '/content/sample_data/adharphoto.jpg'



In [28]:
import cv2
import numpy as np

def straighten_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise and improve line detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection to find edges
    edges = cv2.Canny(blurred, 50, 150, apertureSize=3)

    # Perform Hough line detection
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

    # Find the angle of the most prominent line
    angles = []
    for line in lines:
        rho, theta = line[0]
        angle = theta * 180 / np.pi
        angles.append(angle)
    dominant_angle = np.median(angles)

    # Get image dimensions
    height, width = image.shape[:2]

    # Define the rotation center
    center = (width // 2, height // 2)

    # Perform the rotation to straighten the image
    rotation_matrix = cv2.getRotationMatrix2D(center, dominant_angle, 1.0)
    straightened = cv2.warpAffine(image, rotation_matrix, (width, height), flags=cv2.INTER_LINEAR)

    return straightened




In [29]:
image_processed = straighten_image(image_path)
def extract_text_from_image(image_processed):
    # Open the image file
    image = Image.open(image_path)

    # Perform OCR using Tesseract
    text = pytesseract.image_to_string(image)

    return text

# Example usage

text = extract_text_from_image(image_path)
print(text)

 

Rishav De
DOB: 02/09/2002
MALE
Mobile No: 8697775675

 

9177 0030 2460

VID : 9190 S077 2332 3353

_ MERA AADHAAR, MERI PEHCHAN

 



In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm



In [32]:
import spacy

# Load the pre-trained SpaCy model
nlp = spacy.load("en_core_web_sm")

def extract_person_names(document):
    # Process the document with SpaCy
    doc = nlp(document)

    # Extract person names
    person_names = []
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            person_names.append(ent.text)

    return person_names

def extract_person_names(document, pattern):
    # Process the document with SpaCy
    doc = nlp(document)

    # Compile the regular expression pattern
    regex = re.compile(pattern)

    # Extract person names matching the pattern
    person_names = []
    for ent in doc.ents:
        if ent.label_ == "PERSON" and regex.match(ent.text):
            person_names.append(ent.text)

    return person_names


print(names)
name_regex =  r"(?<![a-zA-Z'])\b[A-Z][a-z']* [A-Z][a-zA-Z']*\b(?![a-zA-Z'])"
# Extract name
name_match = re.search(name_regex,text)
name = name_match.group(0) if name_match else None

names = extract_person_names(text, name_regex)
#print(name)


['Rishav De\nDOB']
Rishav De


In [33]:
import re

# Extracted text
extracted_text = text

# Define regular expressions for each category
name_regex =  r"(?<![a-zA-Z'])\b[A-Z][a-z']* [A-Z][a-zA-Z']*\b(?![a-zA-Z'])"

dob_regex = r"\b(\d{2}/\d{2}/\d{4})\b"
phone_regex = r"(\d{10})"
aadhaar_regex = r"(?<![0-9])(\d{4} \d{4} \d{4})(?![0-9])"
gender_regex = r"\b(Male|Female|FEMALE|MALE)\b"



# Extract DOB
dob_match = re.search(dob_regex, extracted_text)
dob = dob_match.group(1) if dob_match else None

# Extract phone number
phone_match = re.search(phone_regex, extracted_text)
phone_number = phone_match.group(1) if phone_match else None

# Extract Aadhaar number
aadhaar_match = re.search(aadhaar_regex, extracted_text)
aadhaar_number = aadhaar_match.group(1) if aadhaar_match else None

# Extract gender
gender_match = re.search(gender_regex, extracted_text)
gender = gender_match.group(0) if gender_match else None

# Create a dictionary of the extracted data
data = {
    "Name": name,
    "DOB": dob,
    "Phone Number": phone_number,
    "Aadhaar Number": aadhaar_number,
    "Gender": gender
}

# Remove empty values from the dictionary
data = {key: value for key, value in data.items() if value}

# Print the extracted information
for key, value in data.items():
    print(key + ":", value)


Name: Rishav De
DOB: 02/09/2002
Phone Number: 8697775675
Aadhaar Number: 9177 0030 2460
Gender: MALE
